In [1]:
import numpy as np
import matplotlib.pyplot as plt
import SiPMWaveGen as swg
import scipy.signal as scisig
import CFDHitFinder as cfd

In [3]:
timeSample = 0.2  # nanosec
nSamples = 1024  # no. of sampling points in a trigger
speAmplitude = 0.15  # single photon-electron response amplitude in volt
noiseSigmaInVolt = speAmplitude / 15 # noise sigma
riseTime = 1.5  # nanosec
fallTime = 3  # nanosec
nBits = 12  # 12bit ADC
voltMin = -0.9  # minimum voltage ADC in volt
dynamicRange = 1  # dynamic range of ADC
offset = 2000  # offset ADC

cfdThreshold = 0.2  # CFD threshold
durationTheshold = 10  # hit finder alg param: duration of a detected peak to be considered a hit
adjDurationThreshold = 10  # hit finder alg param: duration between two detected peaks to be considered a single hit
nNoiseSigmaThreshold = 1.5  # adcThreshold = (noise threshold) * nNoiseSigmaThreshold + baseline
sgFilter = True  # using Golay-Savitzky filter 
sgWindow = 15  # sg sliding window
sgPolyOrder = 3  # sg smoothing polynomial order

for i in range(10):
    [t, p, true_p] = swg.aDigitizedTrigger(dt=timeSample,
                                           nsamples=nSamples,
                                           speAmplitude=speAmplitude,
                                           noiseSigmaInVolt=noiseSigmaInVolt,
                                           riseTime=riseTime,
                                           fallTime=fallTime,
                                           nBits=nBits,
                                           voltMin=voltMin,
                                           dynamicRange=dynamicRange,
                                           offset=offset)

    [hitList, hitLogic, baseline, noiseSigma] = cfd.HitFinder(p=p,
                                                              noiseSigmaInVolt=speAmplitude/15,
                                                              cfdThreshold=cfdThreshold,
                                                              durationTheshold=durationTheshold,
                                                              adjDurationThreshold=adjDurationThreshold,
                                                              nNoiseSigmaThreshold=nNoiseSigmaThreshold,
                                                              sgFilter=sgFilter,
                                                              sgWindow=sgWindow,
                                                              sgPolyOrder=sgPolyOrder)

    filter_p = scisig.savgol_filter(x=p,
                                    window_length=sgWindow,
                                    polyorder=sgPolyOrder)
    if np.size(hitList) > 0:
        plt.plot(t, p, 'r')
        for x in hitList:
            if x < 1024 and x >= 0:
                plt.axvline(x=(x * 0.2))
        plt.show()
